In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [6]:
# download the cats and dogs dataset and unzip them
# after this run, set your data_dir variable: data_dir = 'Cat_Dog_data'
# so all the data is available. so path to your training data becomes `Cat_Dog_data/train`
# path to test is `Cat_Dog_data/test`

!wget -c https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip

# remove existing directories
!rm -r Cat_Dog_data __MACOSX || true
!unzip -qq Cat_Dog_data.zip

--2019-08-12 12:33:46--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.227.3
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.227.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



## get the train and the test data sets while doing Data Augmanatation in order to reduce
## Overfirritng and enrich the data sets

## then insert all into data loaders

In [0]:
data_dir = 'Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor()])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=True)

# Transfer learning: Using densenet121 model

In [0]:
# detect if GPU available 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = models.densenet121(pretrained=True)

# freeze the layers of the model from gradient descent learning
for param in model.parameters():
  param.requires_grad = False
  

## Build the Classifier net and connect it to the end of the downloded model
## using regularization of Dropout

In [0]:
classifier = nn.Sequential(nn.Linear(1024, 512),
                            nn.ReLU(),
                            nn.Dropout(p=0.2),
                            nn.Linear(512, 64),
                            nn.ReLU(),
                            nn.Dropout(p=0.2),
                            nn.Linear(64, 2),
                            nn.LogSoftmax(dim=1))

model.classifier = classifier

## after cros validation, We can see 4 epochs is enogth
## using optimizer of Adam that combining together momentum and Rmsprop abilitties

In [18]:
criteria = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

epochs = 4
model.to(device)
for epoch in range(epochs):
  model.classifier.train()
  for train_batch,(inputs, lables) in enumerate(trainloader):
    #i += 1
    #if train_batch >= 50:
    #  break
    print('\r\b train batch num:',train_batch,end="")
    # Move input and label tensors to the GPU
    inputs, lables = inputs.to(device), lables.to(device)
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criteria(outputs, lables)
    loss.backward()
    optimizer.step()
    
    
  model.eval()
  with torch.no_grad():
    accuracy = 0
    for test_batch,(inputs, lables) in enumerate(testloader):
      #j += 1
      print('\r\b test batch num: ',test_batch,end="")
      #if test_batch >= 39:
      #  continue
      inputs, lables = inputs.to(device), lables.to(device)
      outputs = model.forward(inputs)
      top_p, top_class = outputs.topk(1, dim=1)
      equality = top_class == lables.view(*top_class.shape)
      accuracy += equality.type(torch.FloatTensor).mean()
    else:
      print(f"\nThe accuray is: {(accuracy.item() / len(testloader)) * 100}%\n")
      

 test batch num:  9
The accuray is: 94.29448127746582%

 test batch num:  9
The accuray is: 95.75175285339355%

 test batch num:  9
The accuray is: 92.58849143981934%

 test batch num:  9
The accuray is: 95.82669258117676%

